In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.base import clone
from itertools import combinations
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import sys
from scipy.stats import ttest_ind, zscore
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
import copy
import json

In [3]:
df_2017 = pd.read_csv("../data/afl_results_2017.csv")
#print(df_2017.shape)
df_2018 = pd.read_csv("../data/afl_results_2018.csv")
#print(df_2018.shape)
df_2019 = pd.read_csv("../data/afl_results_2019.csv")
#print(df_2019.shape)
df_2020 = pd.read_csv("../data/afl_results_2020.csv")
#print(df_2020.shape)
df_2021 = pd.read_csv("../data/afl_results_2021.csv")
#print(df_2021.shape)
df_2022 = pd.read_csv("../data/afl_results_2022.csv")
df_all = pd.concat([df_2017, df_2018, df_2019, df_2020, df_2021,df_2022], axis=0)

In [5]:
df_all.tail()

,match.name,match.date,match.status,match.matchId,match.venue,match.utcStartTime,match.homeTeamId,match.awayTeamId,match.round,match.venueLocalStartTime,...,homeTeamScoreChart.rushedBehinds,homeTeamScoreChart.touchedBehinds,awayTeamScoreChart.goals,awayTeamScoreChart.leftBehinds,awayTeamScoreChart.rightBehinds,awayTeamScoreChart.leftPosters,awayTeamScoreChart.rightPosters,awayTeamScoreChart.rushedBehinds,awayTeamScoreChart.touchedBehinds,venue.landOwner
127,Carlton Vs St Kilda,2022-07-01 09:50:00,CONCLUDED,CD_M20220141603,CD_V190,2022-07-01T09:50:00,CD_T30,CD_T130,CD_R202201416,2022-07-01T19:50:00,...,5,0,14,2,4,2,0,1,0,Wurundjeri people of the Kulin Nation
128,Essendon Vs Sydney Swans,2022-07-02 03:45:00,CONCLUDED,CD_M20220141604,CD_V40,2022-07-02T03:45:00,CD_T50,CD_T160,CD_R202201416,2022-07-02T13:45:00,...,2,0,12,8,2,2,0,2,0,Wurundjeri people of the Kulin Nation
129,Adelaide Crows Vs Melbourne,2022-07-02 06:35:00,CONCLUDED,CD_M20220141601,CD_V6,2022-07-02T06:35:00,CD_T10,CD_T90,CD_R202201416,2022-07-02T16:05:00,...,1,0,14,4,1,1,1,3,0,NaN
130,Geelong Cats Vs North Melbourne,2022-07-02 09:25:00,CONCLUDED,CD_M20220141606,CD_V30,2022-07-02T09:25:00,CD_T70,CD_T100,CD_R202201416,2022-07-02T19:25:00,...,6,1,5,0,1,1,0,0,0,Wadawurrung People
131,Gold Coast Suns Vs Collingwood,2022-07-02 09:25:00,CONCLUDED,CD_M20220141607,CD_V81,2022-07-02T09:25:00,CD_T1000,CD_T40,CD_R202201416,2022-07-02T19:25:00,...,7,0,9,4,2,0,3,4,0,NaN


In [6]:
df_all.columns

Index(['match.name', 'match.date', 'match.status', 'match.matchId',
       'match.venue', 'match.utcStartTime', 'match.homeTeamId',
       'match.awayTeamId', 'match.round', 'match.venueLocalStartTime',
       'match.abbr', 'match.twitterHashTag', 'match.homeTeam.name',
       'match.homeTeam.timeZone', 'match.homeTeam.teamId',
       'match.homeTeam.abbr', 'match.homeTeam.nickname', 'match.awayTeam.name',
       'match.awayTeam.timeZone', 'match.awayTeam.teamId',
       'match.awayTeam.abbr', 'match.awayTeam.nickname', 'venue.address',
       'venue.name', 'venue.state', 'venue.timeZone', 'venue.venueId',
       'venue.abbreviation', 'venue.capacity', 'venue.groundDimension',
       'venue.latitude', 'venue.longitude', 'round.name', 'round.year',
       'round.roundId', 'round.abbreviation', 'round.competitionId',
       'round.roundNumber', 'status', 'matchId', 'scoreWorm', 'scoreMap',
       'lastUpdated', 'homeTeamScore.periodScore',
       'homeTeamScore.rushedBehinds', 'homeTeamS

In [27]:
df_1 = df_all[['match.date','match.homeTeam.name','match.awayTeam.name','round.roundNumber','round.year',
        'homeTeamScore.rushedBehinds', 
        'homeTeamScore.minutesInFront',
       'homeTeamScore.matchScore.totalScore', 
        'homeTeamScore.matchScore.goals',
       'homeTeamScore.matchScore.behinds',
       'homeTeamScoreChart.goals', 
        'homeTeamScoreChart.leftBehinds',
       'homeTeamScoreChart.rightBehinds', 
        'homeTeamScoreChart.leftPosters',
       'homeTeamScoreChart.rightPosters', 
       'homeTeamScoreChart.rushedBehinds',
       'homeTeamScoreChart.touchedBehinds',        
        'awayTeamScore.periodScore',
       'awayTeamScore.rushedBehinds', 
        'awayTeamScore.minutesInFront',
       'awayTeamScore.matchScore.totalScore', 
        'awayTeamScore.matchScore.goals',
       'awayTeamScore.matchScore.behinds',
        'awayTeamScoreChart.goals',
       'awayTeamScoreChart.leftBehinds', 
       'awayTeamScoreChart.rightBehinds',
       'awayTeamScoreChart.leftPosters', 
       'awayTeamScoreChart.rightPosters',
       'awayTeamScoreChart.rushedBehinds', 
        'awayTeamScoreChart.touchedBehinds'
]]

df_1['game'] = np.arange(len(df_1))
df_1['home_game'] = 1

df_1 = df_1.rename(columns={'match.homeTeam.name': 'team', 
                            'match.awayTeam.name': 'opponent',
                           
                           
                           })

df_1


C:\Users\Damian Joseph\AppData\Local\Temp\ipykernel_36748\343700997.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['game'] = np.arange(len(df_1))
C:\Users\Damian Joseph\AppData\Local\Temp\ipykernel_36748\343700997.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['home_game'] = 1


,match.date,team,opponent,round.roundNumber,round.year,homeTeamScore.rushedBehinds,homeTeamScore.minutesInFront,homeTeamScore.matchScore.totalScore,homeTeamScore.matchScore.goals,homeTeamScore.matchScore.behinds,...,awayTeamScore.matchScore.behinds,awayTeamScoreChart.goals,awayTeamScoreChart.leftBehinds,awayTeamScoreChart.rightBehinds,awayTeamScoreChart.leftPosters,awayTeamScoreChart.rightPosters,awayTeamScoreChart.rushedBehinds,awayTeamScoreChart.touchedBehinds,game,home_game
0,2017-03-23 08:20:00,Carlton,Richmond,1,2017,1,0,89,14,5,...,12,20,5,3,0,1,3,0,0,1
1,2017-03-24 08:50:00,Collingwood,Western Bulldogs,1,2017,3,4,86,12,14,...,10,15,3,2,0,1,4,0,1,1
2,2017-03-25 05:35:00,St Kilda,Melbourne,1,2017,2,32,90,13,12,...,12,18,4,5,0,1,2,0,2,1
3,2017-03-25 05:35:00,Sydney Swans,Port Adelaide,1,2017,1,32,82,12,10,...,8,17,1,6,0,0,0,1,3,1
4,2017-03-25 08:25:00,Essendon,Hawthorn,1,2017,2,85,116,17,14,...,19,12,5,9,0,0,5,0,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,2022-07-01 09:50:00,Carlton,St Kilda,16,2022,5,17,78,10,18,...,9,14,2,4,2,0,1,0,1117,1
128,2022-07-02 03:45:00,Essendon,Sydney Swans,16,2022,2,37,95,15,5,...,14,12,8,2,2,0,2,0,1118,1
129,2022-07-02 06:35:00,Adelaide Crows,Melbourne,16,2022,1,4,65,10,5,...,10,14,4,1,1,1,3,0,1119,1
130,2022-07-02 09:25:00,Geelong Cats,North Melbourne,16,2022,6,120,144,21,18,...,2,5,0,1,1,0,0,0,1120,1


In [24]:
df_2 = df_1.rename(columns={'match.homeTeam.name': 'match.awayTeam.name', 'match.awayTeam.name': 'match.homeTeam.name'})
df_2    


,match.date,match.awayTeam.name,match.homeTeam.name,round.roundNumber,round.year,homeTeamScore.rushedBehinds,homeTeamScore.minutesInFront,homeTeamScore.matchScore.totalScore,homeTeamScore.matchScore.goals,homeTeamScore.matchScore.behinds,...,awayTeamScore.matchScore.behinds,awayTeamScoreChart.goals,awayTeamScoreChart.leftBehinds,awayTeamScoreChart.rightBehinds,awayTeamScoreChart.leftPosters,awayTeamScoreChart.rightPosters,awayTeamScoreChart.rushedBehinds,awayTeamScoreChart.touchedBehinds,game,home_game
0,2017-03-23 08:20:00,Carlton,Richmond,1,2017,1,0,89,14,5,...,12,20,5,3,0,1,3,0,0,1
1,2017-03-24 08:50:00,Collingwood,Western Bulldogs,1,2017,3,4,86,12,14,...,10,15,3,2,0,1,4,0,1,1
2,2017-03-25 05:35:00,St Kilda,Melbourne,1,2017,2,32,90,13,12,...,12,18,4,5,0,1,2,0,2,1
3,2017-03-25 05:35:00,Sydney Swans,Port Adelaide,1,2017,1,32,82,12,10,...,8,17,1,6,0,0,0,1,3,1
4,2017-03-25 08:25:00,Essendon,Hawthorn,1,2017,2,85,116,17,14,...,19,12,5,9,0,0,5,0,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,2022-07-01 09:50:00,Carlton,St Kilda,16,2022,5,17,78,10,18,...,9,14,2,4,2,0,1,0,1117,1
128,2022-07-02 03:45:00,Essendon,Sydney Swans,16,2022,2,37,95,15,5,...,14,12,8,2,2,0,2,0,1118,1
129,2022-07-02 06:35:00,Adelaide Crows,Melbourne,16,2022,1,4,65,10,5,...,10,14,4,1,1,1,3,0,1119,1
130,2022-07-02 09:25:00,Geelong Cats,North Melbourne,16,2022,6,120,144,21,18,...,2,5,0,1,1,0,0,0,1120,1


df_2